In [1]:
import pandas as pd
import duckdb
import altair as alt

In [2]:
# build connection ke duckdb
conn = duckdb.connect('../data/instacart.db')

In [3]:
def run_query(query):
    return conn.execute(query).fetchdf()

In [4]:
print("Tables information: ", run_query("SHOW TABLES"))

Tables information:                      name
0                 aisles
1            departments
2  order_products__prior
3  order_products__train
4                 orders
5               products


explore `orders` table

In [9]:
sample_orders = run_query("SELECT * FROM orders LIMIT 10")
print("orders: ")
sample_orders

orders: 


,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0
5,3367565,1,prior,6,2,7,19.0
6,550135,1,prior,7,1,9,20.0
7,3108588,1,prior,8,1,14,14.0
8,2295261,1,prior,9,1,16,0.0
9,2550362,1,prior,10,4,8,30.0


In [21]:
orders_stats = run_query(
    """
    SELECT
        COUNT(*) AS total_orders,
        COUNT(DISTINCT order_id) AS unique_order_id,
        COUNT(DISTINCT user_id) AS unique_users,
        COUNT(DISTINCT eval_set) AS unique_eval_set,
        ROUND(AVG(days_since_prior_order), 2) AS avg_days_between_orders,
        ROUND(AVG(order_hour_of_day), 2) AS avg_order_hour_of_day,
    FROM orders
"""
)

orders_stats

,total_orders,unique_order_id,unique_users,unique_eval_set,avg_days_between_orders,avg_order_hour_of_day
0,3421083,3421083,206209,3,11.11,13.45


In [20]:
order_nulls = run_query(
    """
    SELECT 
        (COUNT(*) - COUNT(order_id)) * 100.0 / COUNT(*) AS order_id_null_perc,
        (COUNT(*) - COUNT(user_id)) * 100.0 / COUNT(*) AS user_id_null_perc,
        (COUNT(*) - COUNT(eval_set)) * 100.0 / COUNT(*) AS eval_set_null_perc,
        (COUNT(*) - COUNT(order_number)) * 100.0 / COUNT(*) AS order_number_null_perc,
        (COUNT(*) - COUNT(order_dow)) * 100.0 / COUNT(*) AS order_dow_null_perc,
        (COUNT(*) - COUNT(order_hour_of_day)) * 100.0 / COUNT(*) AS order_hour_of_day_null_perc,
        (COUNT(*) - COUNT(days_since_prior_order)) * 100.0 / COUNT(*) AS days_since_prior_order_null_perc
    FROM orders
"""
)
order_nulls

,order_id_null_perc,user_id_null_perc,eval_set_null_perc,order_number_null_perc,order_dow_null_perc,order_hour_of_day_null_perc,days_since_prior_order_null_perc
0,0.0,0.0,0.0,0.0,0.0,0.0,6.027594


In [24]:
avg_orders = run_query(
    """
    SELECT
        ROUND(AVG(order_count), 2) AS avg_orders_per_cust,
        MEDIAN(order_count) AS median_orders_per_cust,
        MIN(order_count) AS min_orders,
        MAX(order_count) AS max_orders,
    FROM (
        SELECT user_id, COUNT(*) as order_count
        FROM orders
        GROUP BY user_id
    )
"""
)

avg_orders

,avg_orders_per_cust,median_orders_per_cust,min_orders,max_orders
0,16.59,10.0,4,100
